# Module 3: Simple RNN Practice

## Practice: Creating a Simple RNN for Temperature Prediction

In this example, we will use a simple Recurrent Neural Network (RNN) to predict daily temperatures based on past records. RNNs are useful for time-series forecasting, such as weather prediction, stock market analysis, and trend detection.


## 1. Importing Required Libraries

We import NumPy for numerical operations, TensorFlow/Keras to build and train the RNN, Matplotlib for visualization, and Scikit-learn for scaling.


In this cell we import the libraries required for working with RNNs:

- **numpy**: For numerical operations with arrays.
- **tensorflow** and **keras**: To define and train the RNN.
- **matplotlib.pyplot**: For data visualization.
- **sklearn.preprocessing.MinMaxScaler**: For scaling the data.

A print statement confirms successful imports.

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Input, SimpleRNN, Dense
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

print("Libraries imported successfully!")

## 2. Load and Visualize Temperature Data

We load the daily temperature data from CSV and plot it to observe seasonal patterns.


In this cell we load and explore the daily temperature dataset:

1. Use **pandas** to read the `daily_temperature.csv` file.
2. Extract the `day` and `temperature` columns.
3. Plot the temperature over time to visualize the time series.

In [ ]:
# Load dataset
df = pd.read_csv('./data/daily_temperature.csv')
days = df['day'].values
temperature = df['temperature'].values

# Plot
plt.figure(figsize=(10,4))
plt.plot(days, temperature, label='Temperature')
plt.xlabel('Day')
plt.ylabel('Temperature (°C)')
plt.title('Synthetic Daily Temperature (1 year)')
plt.legend()
plt.show()

## 3. Preparing the Data for the RNN

We normalize the temperatures, create sequences of the past 7 days as input, and the next day's temperature as output. Then split into train/test sets (80/20).


In this cell we preprocess the data:

1. **Scaling**: Transform temperature values to the [0,1] range using `MinMaxScaler`.
2. **Sequence creation**: With `sequence_length=7`, create (X, y) pairs where X contains the previous 7 days and y the next day.
3. **Train-test split**: Allocate 80% for training and 20% for testing.

Print the number of training and testing samples.

In [ ]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0,1))
temp_scaled = scaler.fit_transform(temperature.reshape(-1,1))

# Create sequences
sequence_length = 7
X, y = [], []
for i in range(len(temp_scaled) - sequence_length):
    X.append(temp_scaled[i:i+sequence_length])
    y.append(temp_scaled[i+sequence_length])
X, y = np.array(X), np.array(y)

# Split into train/test
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print(f"Training samples: {X_train.shape[0]}, Testing samples: {X_test.shape[0]}")

## 4. Building and Training the RNN Model

We build a Sequential model with one SimpleRNN layer and one Dense output. We compile with Adam optimizer and MSE loss, then train for 50 epochs.


In this cell we define, compile, and train the RNN model:

- Start with a Keras **Sequential** model.  
- Use an explicit **Input** layer to specify the sequence shape (sequence_length time steps, 1 feature).  
- Add a **SimpleRNN** layer with 10 units and **ReLU** activation.  
- Finish with a **Dense(1)** layer to predict the next continuous value.  
- Compile with the **Adam** optimizer and **MSE** loss, then train for 50 epochs with validation on the test set.

In [ ]:
# Define the sequence length as used in preprocessing
sequence_length = 7

model = Sequential([
    Input(shape=(sequence_length, 1)),
    SimpleRNN(10, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), verbose=1)

## 5. Making Predictions

We predict on the test set, inverse-transform the results, and compare predicted vs actual temperatures.


In this cell we evaluate the model's predictions:

1. Call `model.predict` on `X_test` and inverse-transform the scaled values.
2. Rescale both predictions and true values to Celsius.
3. Plot actual vs. predicted temperature using solid and dashed lines.
4. Add labels, title, and legend for clarity.

In [ ]:
# Predict
predictions = model.predict(X_test)
pred_rescaled = scaler.inverse_transform(predictions)
y_test_rescaled = scaler.inverse_transform(y_test)

# Plot results
plt.figure(figsize=(10,4))
plt.plot(y_test_rescaled, label='Actual Temperature')
plt.plot(pred_rescaled, label='Predicted Temperature', linestyle='dashed')
plt.xlabel('Time Step')
plt.ylabel('Temperature (°C)')
plt.title('Temperature Prediction Using RNN')
plt.legend()
plt.show()